In [1]:
# DISCLAIMER: Large part of this code for training new entities based on a pre-trained model, 
# is taken from Isaac Aderogba under the following link https://deepnote.com/publish/2cc2d19c-c3ac-4321-8853-0bcf2ef565b3
# 
# Statistics Canton Zurich humbly tried to adapt his code to our purposes of training
#statistically relevant entities

In [2]:
from __future__ import print_function, unicode_literals
import pandas as pd
import numpy as np
from random import sample
import io, csv
import re
import random
import json

In [3]:
def uniform_cleaning(str_in):
    str_in=str_in.replace("-","")
    str_in=str_in.replace("(","")
    str_in=str_in.replace(")","")
    str_in=str_in.replace("ü","ue")
    str_in=str_in.replace("ä","ae")
    str_in=str_in.replace("ö","oe")
    return(str_in)
    
    

In [7]:
random.seed(1)

df=pd.read_csv("data/datasets_overview.csv")

#one more prepatation
df['question_type'] = df['dataset_title'].str.extract(r"\[(.*?)\]", expand=False)
df['question_type'] = np.where(df['question_type']== '%', "percent", "cardinal")
df['dataset_title'] = df['dataset_title'].str.replace(r"\[(.*?)\]", "")
#structurize relevant informations such as variables

#load content of datasets



sentences=pd.read_csv("input/question_generator.csv")
out_sentences=[]

for i in df['index'].unique():
    filename="data/"+str(i)+".csv"
    with io.open(filename, 'r', encoding="latin-1") as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.readline(), [',',';'])
        csvfile.seek(0)
        mydelimiter=dialect.delimiter
    this_data=pd.read_csv(filename,delimiter=dialect.delimiter)
    vars=df['var'].loc[df['index'] == i]
    dataset_title=df['dataset_title'].loc[df['index']==i].any()
    question_type=df['question_type'].loc[df['index']==i].any()
    if vars.str.contains("INDIKATOR_VALUE").sum():
        main=uniform_cleaning(dataset_title.strip())
        filter_vars=vars
        #print(filter_vars)
    else:
        continue #temporary so that all variables look alike
        print("ATTRIBUTING MAIN VARIABLE TO: ",dataset_title)
        length_title_string=len(dataset_title.split())
        highest_similarity=0
        which=None
        for var in vars:
            temp_string=dataset_title+" "+var
            #print(temp_string)
            doc = nlp(temp_string)
            assert len(doc.vector) == len(doc[0].vector)
            calc_similarity=doc[:length_title_string].similarity(doc[length_title_string:])
            if  calc_similarity> highest_similarity:
                highest_similarity=calc_similarity
                which=var

        print("Highest similarity:",var,highest_similarity)
        main=uniform_cleaning(var)
        vars=vars.tolist()
        print(vars)
        print(main)
        filter_vars=vars.remove(main)
    #the following temporary because it is standardized
    filter_vars=list(filter_vars)
    filter_vars.append("")
    try:
        filter_vars.remove("INDIKATOR_JAHR")
        filter_vars.remove("GEBIET_NAME")
        filter_vars.remove("BFS_NR")
        filter_vars.remove("INDIKATOR_VALUE")
    except:
        print("variable removing empty")

    for sentence in sentences['question'].loc[sentences['main_type'] == question_type]:
        out_entities = []
        
        sentence=sentence.replace("{main}",main)
        sentence=sentence.replace("{localitylevel}","")#at the moment empty
        #TODO either one locality, one level, or several localities
        random_value=sample(["one locality","one level","several localities"],1)[0]
        if random_value=="one locality":
            locality_insert="in "+sample(list(this_data['GEBIET_NAME']),1)[0]
        if random_value=="one level":
            locality_insert=sample(["für den gesamten Kanton","im Kanton Zürich","auf Bezirksebene",
            "für alle Bezirke","pro Bezirk","auf Gemeindeebene","für alle Gemeinden","pro Gemeinde"],1)[0]
        if random_value=="several localities":
            locality_insert=""
            local_loop=sample([1,2,3],1)[0]
            for local in range(0,local_loop):
                if local!=0 and local!=(local_loop-1):
                    locality_insert+=", "
                if local!=0 and local==(local_loop-1):
                    locality_insert+=" und "
                locality_insert+=sample(list(this_data['GEBIET_NAME']),1)[0]
        sentence=sentence.replace("{locality}",locality_insert)
        sentence=sentence.replace("{yeartime}","")#TODO no,aktuellste,neuste, value from list, from to year
        sentence=sentence.replace("{filter}",sample(list(filter_vars),1)[0])

        for mat in re.findall(r'.*?\[(.*)].*', sentence):
            which_part=sample([1,2],1)

            if which_part==1:
                sentence=sentence.replace("["+mat+"]",mat.partition("|")[0])
                #sentence=re.sub("["+mat+"]",mat.partition("|")[0],sentence)
            else:
                sentence=sentence.replace("["+mat+"]",mat.partition("|")[2])
                #sentence=re.sub("["+mat+"]",mat.partition("|")[2],sentence)
                
        
        #now the symbol - has to be deleted as it gives issues
        sentence=uniform_cleaning(sentence)

        

        
        #### TAGGING

        #1) GRAN
        match_span = re.search(r'\bGEMEINDE\b|\bGEMEINDEN\b|\bGEMEINDEEBENE\b|\bBEZIRK\b|\bBEZIRKSEBENE\b|\bBEZIRKE\b|\bKANTON\b|\bKANTONSEBENE\b|\bREGION\b', sentence,flags=re.IGNORECASE)
        if match_span is not None:
            match_span=match_span.span()
            out_entities.append((match_span[0], match_span[1], "GRAN"))
            
        #2) DATASET
        print(main)
        match_span = re.search(main,sentence,flags=re.IGNORECASE) 
        print(match_span)
        if match_span is not None:
            match_span=match_span.span()
            out_entities.append((match_span[0], match_span[1], "DATA"))
            
            
        

        out_sentences.append((sentence, {"entities": out_entities}))
        
        
        
       
                
       

<ipython-input-7-ec3aa7fda5b6>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['dataset_title'] = df['dataset_title'].str.replace(r"\[(.*?)\]", "")


Eigenkapital
<re.Match object; span=(9, 21), match='Eigenkapital'>
Eigenkapital
<re.Match object; span=(16, 28), match='Eigenkapital'>
Eigenkapital
<re.Match object; span=(12, 24), match='Eigenkapital'>
Eigenkapital
<re.Match object; span=(11, 23), match='Eigenkapital'>
Bruttoverschuldungsanteil
<re.Match object; span=(22, 47), match='Bruttoverschuldungsanteil'>
Bruttoverschuldungsanteil
<re.Match object; span=(12, 37), match='Bruttoverschuldungsanteil'>
Steuerbares Vermoegen natuerliche Pers.
<re.Match object; span=(9, 48), match='Steuerbares Vermoegen natuerliche Pers.'>
Steuerbares Vermoegen natuerliche Pers.
<re.Match object; span=(16, 55), match='Steuerbares Vermoegen natuerliche Pers.'>
Steuerbares Vermoegen natuerliche Pers.
<re.Match object; span=(12, 51), match='Steuerbares Vermoegen natuerliche Pers.'>
Steuerbares Vermoegen natuerliche Pers.
<re.Match object; span=(11, 50), match='Steuerbares Vermoegen natuerliche Pers.'>
Sterbefaelle
<re.Match object; span=(9, 21), match='St

Steuerertrag von juristischen Personen
<re.Match object; span=(9, 47), match='Steuerertrag von juristischen Personen'>
Steuerertrag von juristischen Personen
<re.Match object; span=(16, 54), match='Steuerertrag von juristischen Personen'>
Steuerertrag von juristischen Personen
<re.Match object; span=(12, 50), match='Steuerertrag von juristischen Personen'>
Steuerertrag von juristischen Personen
<re.Match object; span=(11, 49), match='Steuerertrag von juristischen Personen'>
Gebaeudevolumen
<re.Match object; span=(9, 24), match='Gebaeudevolumen'>
Gebaeudevolumen
<re.Match object; span=(16, 31), match='Gebaeudevolumen'>
Gebaeudevolumen
<re.Match object; span=(12, 27), match='Gebaeudevolumen'>
Gebaeudevolumen
<re.Match object; span=(11, 26), match='Gebaeudevolumen'>
Rinderviehbestand
<re.Match object; span=(9, 26), match='Rinderviehbestand'>
Rinderviehbestand
<re.Match object; span=(16, 33), match='Rinderviehbestand'>
Rinderviehbestand
<re.Match object; span=(12, 29), match='Rinderviehbes

Anteil 6+ Zi.Wohnungen
None
Anteil 6+ Zi.Wohnungen
None
MotorradNeuzulasssungen pro 1000 Einw.
<re.Match object; span=(9, 47), match='MotorradNeuzulasssungen pro 1000 Einw.'>
MotorradNeuzulasssungen pro 1000 Einw.
<re.Match object; span=(16, 54), match='MotorradNeuzulasssungen pro 1000 Einw.'>
MotorradNeuzulasssungen pro 1000 Einw.
<re.Match object; span=(12, 50), match='MotorradNeuzulasssungen pro 1000 Einw.'>
MotorradNeuzulasssungen pro 1000 Einw.
<re.Match object; span=(11, 49), match='MotorradNeuzulasssungen pro 1000 Einw.'>
Neuzulassungen anderer Antrieb
<re.Match object; span=(22, 52), match='Neuzulassungen anderer Antrieb'>
Neuzulassungen anderer Antrieb
<re.Match object; span=(12, 42), match='Neuzulassungen anderer Antrieb'>
Bestand Benzinmotor
<re.Match object; span=(22, 41), match='Bestand Benzinmotor'>
Bestand Benzinmotor
<re.Match object; span=(12, 31), match='Bestand Benzinmotor'>
Bestand anderer Antrieb
<re.Match object; span=(22, 45), match='Bestand anderer Antrieb'>
Bes

Anteil GK F
<re.Match object; span=(22, 33), match='Anteil GK F'>
Anteil GK F
<re.Match object; span=(12, 23), match='Anteil GK F'>
Verkaeufe EFH
<re.Match object; span=(9, 22), match='Verkaeufe EFH'>
Verkaeufe EFH
<re.Match object; span=(16, 29), match='Verkaeufe EFH'>
Verkaeufe EFH
<re.Match object; span=(12, 25), match='Verkaeufe EFH'>
Verkaeufe EFH
<re.Match object; span=(11, 24), match='Verkaeufe EFH'>
Landwirtschaftsbetriebe
<re.Match object; span=(9, 32), match='Landwirtschaftsbetriebe'>
Landwirtschaftsbetriebe
<re.Match object; span=(16, 39), match='Landwirtschaftsbetriebe'>
Landwirtschaftsbetriebe
<re.Match object; span=(12, 35), match='Landwirtschaftsbetriebe'>
Landwirtschaftsbetriebe
<re.Match object; span=(11, 34), match='Landwirtschaftsbetriebe'>
Nettoaufwand Umweltschutz und Raumordnung
<re.Match object; span=(9, 50), match='Nettoaufwand Umweltschutz und Raumordnung'>
Nettoaufwand Umweltschutz und Raumordnung
<re.Match object; span=(16, 57), match='Nettoaufwand Umweltschu

Nettoaufwand Bildung
<re.Match object; span=(11, 31), match='Nettoaufwand Bildung'>
Bevoelkerung: Maenner
<re.Match object; span=(9, 30), match='Bevoelkerung: Maenner'>
Bevoelkerung: Maenner
<re.Match object; span=(16, 37), match='Bevoelkerung: Maenner'>
Bevoelkerung: Maenner
<re.Match object; span=(12, 33), match='Bevoelkerung: Maenner'>
Bevoelkerung: Maenner
<re.Match object; span=(11, 32), match='Bevoelkerung: Maenner'>
Anteil 3Personenhaushalte
<re.Match object; span=(22, 47), match='Anteil 3Personenhaushalte'>
Anteil 3Personenhaushalte
<re.Match object; span=(12, 37), match='Anteil 3Personenhaushalte'>
Anteil Vollzeitbeschaeftigte Landwirtschaft
<re.Match object; span=(22, 65), match='Anteil Vollzeitbeschaeftigte Landwirtschaft'>
Anteil Vollzeitbeschaeftigte Landwirtschaft
<re.Match object; span=(12, 55), match='Anteil Vollzeitbeschaeftigte Landwirtschaft'>
KRW Waehleranteil EDU
<re.Match object; span=(22, 43), match='KRW Waehleranteil EDU'>
KRW Waehleranteil EDU
<re.Match object;

Bauinv. Spital, Kultur usw. 5 J.
<re.Match object; span=(9, 41), match='Bauinv. Spital, Kultur usw. 5 J.'>
Bauinv. Spital, Kultur usw. 5 J.
<re.Match object; span=(16, 48), match='Bauinv. Spital, Kultur usw. 5 J.'>
Bauinv. Spital, Kultur usw. 5 J.
<re.Match object; span=(12, 44), match='Bauinv. Spital, Kultur usw. 5 J.'>
Bauinv. Spital, Kultur usw. 5 J.
<re.Match object; span=(11, 43), match='Bauinv. Spital, Kultur usw. 5 J.'>
Bauinvestitionen
<re.Match object; span=(9, 25), match='Bauinvestitionen'>
Bauinvestitionen
<re.Match object; span=(16, 32), match='Bauinvestitionen'>
Bauinvestitionen
<re.Match object; span=(12, 28), match='Bauinvestitionen'>
Bauinvestitionen
<re.Match object; span=(11, 27), match='Bauinvestitionen'>
Bauzonen ueberbaut pro Kopf
<re.Match object; span=(9, 36), match='Bauzonen ueberbaut pro Kopf'>
Bauzonen ueberbaut pro Kopf
<re.Match object; span=(16, 43), match='Bauzonen ueberbaut pro Kopf'>
Bauzonen ueberbaut pro Kopf
<re.Match object; span=(12, 39), match='Bau

Ø Reingewinn steuerpfl. jur. Pers.
<re.Match object; span=(9, 43), match='Ø Reingewinn steuerpfl. jur. Pers.'>
Ø Reingewinn steuerpfl. jur. Pers.
<re.Match object; span=(16, 50), match='Ø Reingewinn steuerpfl. jur. Pers.'>
Ø Reingewinn steuerpfl. jur. Pers.
<re.Match object; span=(12, 46), match='Ø Reingewinn steuerpfl. jur. Pers.'>
Ø Reingewinn steuerpfl. jur. Pers.
<re.Match object; span=(11, 45), match='Ø Reingewinn steuerpfl. jur. Pers.'>
Beschaeftigte im Sekundaersektor
<re.Match object; span=(9, 41), match='Beschaeftigte im Sekundaersektor'>
Beschaeftigte im Sekundaersektor
<re.Match object; span=(16, 48), match='Beschaeftigte im Sekundaersektor'>
Beschaeftigte im Sekundaersektor
<re.Match object; span=(12, 44), match='Beschaeftigte im Sekundaersektor'>
Beschaeftigte im Sekundaersektor
<re.Match object; span=(11, 43), match='Beschaeftigte im Sekundaersektor'>
Nettoaufwand Gesundheit
<re.Match object; span=(9, 32), match='Nettoaufwand Gesundheit'>
Nettoaufwand Gesundheit
<re.Match

Nettoaufwand allgemeine Verwaltung
<re.Match object; span=(9, 43), match='Nettoaufwand allgemeine Verwaltung'>
Nettoaufwand allgemeine Verwaltung
<re.Match object; span=(16, 50), match='Nettoaufwand allgemeine Verwaltung'>
Nettoaufwand allgemeine Verwaltung
<re.Match object; span=(12, 46), match='Nettoaufwand allgemeine Verwaltung'>
Nettoaufwand allgemeine Verwaltung
<re.Match object; span=(11, 45), match='Nettoaufwand allgemeine Verwaltung'>
Personenwagen
<re.Match object; span=(9, 22), match='Personenwagen'>
Personenwagen
<re.Match object; span=(16, 29), match='Personenwagen'>
Personenwagen
<re.Match object; span=(12, 25), match='Personenwagen'>
Personenwagen
<re.Match object; span=(11, 24), match='Personenwagen'>
Grosse Betriebe 250+ VZÄ
None
Grosse Betriebe 250+ VZÄ
None
Grosse Betriebe 250+ VZÄ
None
Grosse Betriebe 250+ VZÄ
None
VZÄ im Sekundaersektor
<re.Match object; span=(9, 31), match='VZÄ im Sekundaersektor'>
VZÄ im Sekundaersektor
<re.Match object; span=(16, 38), match='VZÄ 

Schuel. Sekundarstufe II
<re.Match object; span=(9, 33), match='Schuel. Sekundarstufe II'>
Schuel. Sekundarstufe II
<re.Match object; span=(16, 40), match='Schuel. Sekundarstufe II'>
Schuel. Sekundarstufe II
<re.Match object; span=(12, 36), match='Schuel. Sekundarstufe II'>
Schuel. Sekundarstufe II
<re.Match object; span=(11, 35), match='Schuel. Sekundarstufe II'>
Neuerstellte MFHWohnungen
<re.Match object; span=(9, 34), match='Neuerstellte MFHWohnungen'>
Neuerstellte MFHWohnungen
<re.Match object; span=(16, 41), match='Neuerstellte MFHWohnungen'>
Neuerstellte MFHWohnungen
<re.Match object; span=(12, 37), match='Neuerstellte MFHWohnungen'>
Neuerstellte MFHWohnungen
<re.Match object; span=(11, 36), match='Neuerstellte MFHWohnungen'>
Flaeche
<re.Match object; span=(9, 16), match='Flaeche'>
Flaeche
<re.Match object; span=(16, 23), match='Flaeche'>
Flaeche
<re.Match object; span=(12, 19), match='Flaeche'>
Flaeche
<re.Match object; span=(11, 18), match='Flaeche'>
Altersquotient
<re.Match ob

In [8]:
print(out_sentences[:10])
with open("input/tagged_sentences.json","w",encoding='utf-8') as outfile:
    json.dump(out_sentences, outfile, ensure_ascii=False)


[('Wie viel Eigenkapital hat  in Wiesendangen bis 2013  ?', {'entities': [(9, 21, 'DATA')]}), ('Welches ist das Eigenkapital  im Kanton Zuerich ?', {'entities': [(33, 39, 'GRAN'), (16, 28, 'DATA')]}), ('Ich brauche Eigenkapital ', {'entities': [(12, 24, 'DATA')]}), ('Daten fuer Eigenkapital ', {'entities': [(11, 23, 'DATA')]}), ('Was ist der Anteil an Bruttoverschuldungsanteil  fuer alle Gemeinden ?', {'entities': [(59, 68, 'GRAN'), (22, 47, 'DATA')]}), ('Ich brauche Bruttoverschuldungsanteil in Prozent  fuer alle Bezirke ', {'entities': [(60, 67, 'GRAN'), (12, 37, 'DATA')]}), ('Wie viel Steuerbares Vermoegen natuerliche Pers. hat  fuer den gesamten Kanton  ?', {'entities': [(72, 78, 'GRAN'), (9, 48, 'DATA')]}), ('Welches ist das Steuerbares Vermoegen natuerliche Pers.  in Bezirk Buelach ?', {'entities': [(60, 66, 'GRAN'), (16, 55, 'DATA')]}), ('Ich brauche Steuerbares Vermoegen natuerliche Pers. ', {'entities': [(12, 51, 'DATA')]}), ('Daten fuer Steuerbares Vermoegen natuerliche Pers.

In [9]:
print(out_sentences[1])
print(out_sentences[1][0][10])
print(out_sentences[1][0][12-1])

('Welches ist das Eigenkapital  im Kanton Zuerich ?', {'entities': [(33, 39, 'GRAN'), (16, 28, 'DATA')]})
t
 
